In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [17]:
class TemporalStreamConvNet(nn.Module):
    def __init__(self):
        super(TemporalStreamConvNet, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=2, out_channels=96, kernel_size=7, stride=2)
        self.conv2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=2)
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1)
        self.conv4 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1)
        self.conv5 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1)

        # Pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc6 = nn.Linear(in_features=4608, out_features=4096) # Adjust in_features based on the output of conv5
        self.fc7 = nn.Linear(in_features=4096, out_features=2048)

        # Dropout layer
        self.dropout = nn.Dropout()

    def forward(self, x):
        # Apply convolutions, batch normalization, ReLU activations, and pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool(F.relu(self.conv5(x)))

        # Flatten the output for the fully connected layer
        x = torch.flatten(x, 1)

        # Apply fully connected layers with dropout
        x = self.dropout(F.relu(self.fc6(x)))
        x = self.dropout(F.relu(self.fc7(x)))

        return x

In [19]:
# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print("CUDA Available:", cuda_available)

CUDA Available: False


In [20]:
# Instantiate the model
model = TemporalStreamConvNet()
# Move the model to GPU if CUDA is available
if cuda_available:
    model = model.to('cuda')

In [21]:
# Example of using the model with a dummy input
dummy_input = torch.randn(1, 2, 224, 224)  # Example input tensor; adjust the size as needed

# Move the data to GPU if CUDA is available
if cuda_available:
    dummy_input = dummy_input.to('cuda')

# Forward pass
output = model(dummy_input)
print(output)

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0104, 0.0000]],
       grad_fn=<MulBackward0>)
